<a href="https://colab.research.google.com/github/khmelkoff/TraxRuReformer/blob/main/TraxRuChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install trax==1.3.6

     |████████████████████████████████| 471kB 8.6MB/s 


In [2]:
# !pip install --upgrade jaxlib
!pip install --upgrade jax jaxlib==0.1.61+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     |████████████████████████████████| 593kB 7.3MB/s 
     |████████████████████████████████| 155.7MB 85kB/s 
  Created wheel for jax: filename=jax-0.2.10-cp37-none-any.whl size=679776 sha256=b9a925161d76d487dd7a994e6b967e8e248016d9e69f3427bf019e56c24e039d
  Stored in directory: /root/.cache/pip/wheels/44/ea/ac/3be3bc19ee3b62f6fe1561eb6df1199284bb6bab819c1befa4
Successfully built jax
  Found existing installation: jax 0.2.9
    Uninstalling jax-0.2.9:
      Successfully uninstalled jax-0.2.9
  Found existing installation: jaxlib 0.1.60+cuda101
    Uninstalling jaxlib-0.1.60+cuda101:
      Successfully uninstalled jaxlib-0.1.60+cuda101


In [ ]:
!pip list | grep trax

trax                          1.3.7                


In [2]:

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import re
# import sentencepiece as spm

import trax
from trax import layers as tl
from trax.supervised import decoding
from trax.supervised import training

import textwrap
wrapper = textwrap.TextWrapper(width=70)

%matplotlib inline

ImportError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
VOCAB_FILE = 'bpe32.model'
VOCAB_DIR = '/content/drive/MyDrive/'
VOCAB_TYPE = 'sentencepiece'

vocab_size = trax.data.vocab_size(
    vocab_type= VOCAB_TYPE,
    vocab_file= VOCAB_FILE,
    vocab_dir= VOCAB_DIR)

print('vocab size: ', vocab_size)

vocab size:  32000


# Load and clear the dialogues

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/dialogues.tsv', sep='\t')
data.head()

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...


In [ ]:
test = data.dialogue.iloc[0]
print(wrapper.fill(test))

<span class=participant_2>Пользователь 2: Привет) расскажи о
себе</span><br /><span class=participant_1>Пользователь 1: Привет) под
вкусный кофеек настроение поболтать появилось<br />)</span><br /><span
class=participant_2>Пользователь 2: Что читаешь? Мне нравится
классика</span><br /><span class=participant_2>Пользователь 2: Я тоже
люблю пообщаться</span><br /><span class=participant_1>Пользователь 1:
Люблю животных, просто обожаю, как и свою работу)</span><br /><span
class=participant_1>Пользователь 1: Я фантастику люблю</span><br
/><span class=participant_2>Пользователь 2: А я выращиваю
фиалки</span><br /><span class=participant_2>Пользователь 2: И веду
здоровый и активный образ жизни!</span><br /><span
class=participant_1>Пользователь 1: Ух ты, интересно.</span><br
/><span class=participant_2>Пользователь 2: Ты случайно не принц на
белом коне? Я его очень жду<br />..</span><br /><span
class=participant_1>Пользователь 1: А у меня из хобби каждую неделю
тусить с моим лучшим<br />друг

In [ ]:
def dialogue_clearing(string):
    '''
    Clear text from special characters
    Args:
        string (string): one dialog text
    
    Return
        list of senteces
    '''
    clear_dialogue = []
    for s in string.split('</span>'):
        s = re.sub('<span class=participant_1>', '', s)
        s = re.sub('<span class=participant_2>', '', s)
        s = re.sub(r'^<br />', '', s)
        s = re.sub('<br />', ' ', s)
        s = re.sub(' ,', ',', s)
        s = re.sub(' \.', '.', s)
        s = re.sub(',', ', ', s)
        s = re.sub('  ', ' ', s)
        s = re.sub(r'\r\n', ' ', s)
        s = re.sub('"', '', s)
        s = re.sub('Пользователь ', '', s)
        s = re.sub('Пользователь ', '', s)
        
        if len(s) != 0:
            clear_dialogue.append(s)
    
    return clear_dialogue

In [ ]:
for s in dialogue_clearing(test):
    print(s)

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика
Пользователь 2: Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу)
Пользователь 1: Я фантастику люблю
Пользователь 2: А я выращиваю фиалки
Пользователь 2: И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [ ]:
def dialogue_concat(sentences_list):
    '''
    Concatenate several consecutive sentences of one user to one sequence
    
    Args:
        sentences_list (list): list of sentences for one dialogue
        
    Return:
        list of senteces
    '''

    clear_dialogue_concat = [sentences_list[0]]
    for i in range(1, len(sentences_list)):
        s = sentences_list[i]
        if clear_dialogue_concat[-1][:2] == s[:2]:
            clear_dialogue_concat[-1] = clear_dialogue_concat[-1] + ' ' + s[3:] 
        else:
            clear_dialogue_concat.append(s)
            
    return clear_dialogue_concat

In [ ]:
dialogue = dialogue_clearing(test)
for s in dialogue_concat(dialogue):
    print(s)

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 2: А я выращиваю фиалки И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [ ]:
def swap_users(sentence_list):
    '''
    Swap user1 and user2
    
    Args:
        sentences_list (list): list of sentences for one dialogue
        
    Return:
        list of senteces
    '''

    dialogue = []
    for s in sentence_list:
        if s.find('2:') == 0:
            s = re.sub('2:', '1:', s)
            dialogue.append(s)
            continue
    
        if s.find('1:') == 0:
            s = re.sub('1:', '2:', s)
            dialogue.append(s)
            continue
    
    return dialogue

In [ ]:
dialogue = dialogue_clearing(test)
dialogue = dialogue_concat(dialogue)
for s in swap_users(dialogue):
    print(s)

Пользователь 1: Привет) расскажи о себе
Пользователь 2: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 1: Что читаешь? Мне нравится классика Я тоже люблю пообщаться
Пользователь 2: Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 1: А я выращиваю фиалки И веду здоровый и активный образ жизни!
Пользователь 2: Ух ты, интересно.
Пользователь 1: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 2: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [ ]:
untokenized_data = []
dialogue_len = []
tmp = []
for i in tqdm(range(data.shape[0])):
    dialogue = data.dialogue.iloc[i]
    
    dialogue = dialogue_clearing(dialogue)
    dialogue = dialogue_concat(dialogue)
    
    if dialogue[0].find('2:') == 0:
        dialogue = swap_users(dialogue)
    
    tmp.append(dialogue)
    
    # augmentation
    if (len(dialogue[2:]) >= 8): tmp.append(dialogue[2:])
    if (len(dialogue[4:]) >= 8): tmp.append(dialogue[4:])
    if (len(dialogue[6:]) >= 8): tmp.append(dialogue[6:])
    if (len(dialogue[8:]) >= 8): tmp.append(dialogue[8:])
    if (len(dialogue[10:]) >= 8): tmp.append(dialogue[10:])
    if (len(dialogue[12:]) >= 8): tmp.append(dialogue[12:])
    if (len(dialogue[14:]) >= 8): tmp.append(dialogue[14:])
    if (len(dialogue[16:]) >= 8): tmp.append(dialogue[16:])    
        
    for d in tmp:    
        string = ' '.join(d)
        string = string.strip()
        string = string + ' STOP\n'
        dialogue_len.append(len(d))
        untokenized_data.append(string)
        
    tmp.clear()
    
    random.Random(13).shuffle(untokenized_data)
    
with open('dialogues2.txt', 'w', encoding='utf-8') as file:
     file.writelines(untokenized_data)

100%|██████████| 10013/10013 [00:04<00:00, 2358.29it/s]


In [ ]:
print(wrapper.fill(untokenized_data[0]))

 Пользователь 1: Привет) расскажи о себе Пользователь 2: Привет) под
вкусный кофеек настроение поболтать появилось ) Пользователь 1: Что
читаешь? Мне нравится классика Я тоже люблю пообщаться Пользователь 2:
Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 1: А я выращиваю фиалки И веду здоровый и активный образ
жизни! Пользователь 2: Ух ты, интересно. Пользователь 1: Ты случайно
не принц на белом коне? Я его очень жду.. Пользователь 2: А у меня из
хобби каждую неделю тусить с моим лучшим другом)


In [ ]:
# train tokenizer
# spm.SentencePieceTrainer.train('--input=dialogues2.txt --pad_id=0 --bos_id=-1 --eos_id=1 --unk_id=2 \
#                                 --model_prefix=bpe --vocab_size=32000 --model_type=bpe')

# Data preprocessing and create the generator

In [6]:
# loading prepared set to save time

with open('/content/drive/MyDrive/dialogues2.txt', 'r', encoding='utf-8') as file:
     untokenized_data = file.readlines() 

untokenized_data = [s.strip() for s in untokenized_data]

print(wrapper.fill(untokenized_data[0]))

1: А тебя как зовут? 2: Виктор 1: Приятно познакомиться) 2: Чем
увлекаешься? 1: Кем работаешь? Чем увлекаешься? 2: Я таксист и мне
приходится работать по ночам А чем увлекаешься ты? 1: Я спортсмен-
плавец, люблю работать, как над собой, так и в целом А увлечение,
очень вкусно готовлю У тебя есть домашнее животные? 2: Ты большой
молодец, кстати, я увлекаюсь теннисом, как тебе такой вид спорта? Нет,
домашних животных нет 1: Плаванье) Я очень хочу хомяка, а жена не
разрешает Какое кино нравиться? 2: Эээх, очень жаль, ну ничего, может
когда- нибудь ты исполнить свою мечту 1: Хотелось бы) 2: Кино не
люблю, я больше музыку слушаю, в особенности шансон 1: У тебя есть
мечта? А я постоянно хожу в кинотеатры на приметные) 2: Да, конечно, а
у тебя? Есть какой-то любимый жанр? Кстати, я еще люблю читать А ты
любишь книги? 1: Да, мистика Когда все на гране) А какие книги любишь?
2: Ооо, интересно звучит 1: Тоже, всего Булгакова перечитал 2:
Детективы Ооо Здорово 1: Убийство в полярном экспресс чита

In [ ]:
# Attention! do not run
# random.shuffle(untokenized_data)
# print(wrapper.fill(untokenized_data[0]))

In [7]:
# train/eval split
margin = int(len(untokenized_data)*0.95)

train_dialogues = untokenized_data[:margin]
print('train cases: ', len(train_dialogues))
eval_dialogues = untokenized_data[margin:]
print('eval cases: ', len(eval_dialogues))

train cases:  44460
eval cases:  2341


In [ ]:
# clear mem
# del untokenized_data

In [8]:
def data_generator(data, shuffle=True):
    '''
      Input: 
        data - list of dialogues
        shuffle - If True: shuffle the data order
      Output:
        a tuple containing 2 elements:
        article
        summary
    '''
    
    data_lng = len(data) # len(data)
    index_list = [*range(data_lng)] # Create a list with the ordered indexes of sample data

    if shuffle:
        random.shuffle(index_list) # re-shuffle the order
    
    index = 0 # Start with the first element
    while True:
        # Wrap the index each time that we reach the end of the list
        if index >= data_lng:
            index = 0
            if shuffle:
                random.shuffle(index_list) # re-shuffle the order
            
        sample = data[index_list[index]]
        index += 1
        yield(sample, sample) # input and target

# create data streams
def train_data_stream():
    return data_generator(train_dialogues, shuffle=True)

def eval_data_stream():
    return data_generator(eval_dialogues, shuffle=True)

In [9]:
def detokenize(integers):
    s = trax.data.detokenize(
        integers,
        vocab_type=VOCAB_TYPE,
        vocab_file=VOCAB_FILE,
        vocab_dir=VOCAB_DIR) # loading pre-prepared model to save time
    return s


def tokenize(s):
    inputs =  next(trax.data.tokenize(
        iter([s]),
        vocab_type=VOCAB_TYPE,
        vocab_file=VOCAB_FILE,
        vocab_dir=VOCAB_DIR))
    
    return list(inputs)

In [10]:
# trax allows us to use combinators to generate our data pipeline
data_pipeline = trax.data.Serial(
    
    # tokenize the data
    trax.data.Tokenize(vocab_type=VOCAB_TYPE,
                       vocab_dir=VOCAB_DIR,
                       vocab_file=VOCAB_FILE),
    
    # filter too long sequences
    trax.data.FilterByLength(max_length=2048, min_length=64),
    
    # bucket by length
    trax.data.BucketByLength(boundaries=[128, 256, 512, 1024],
                             batch_sizes=[16, 8, 4, 2, 1]),
    
    # add loss weights but do not add it to the padding tokens (i.e. 0)
    trax.data.AddLossWeights(id_to_mask=0)
)

# apply the data pipeline to our train and eval sets
train_stream = data_pipeline(data_generator(train_dialogues))
eval_stream = data_pipeline(data_generator(eval_dialogues))

In [11]:
# the stream generators will yield (input, target, weights). let's just grab the input for inspection
inp, _, m = next(train_stream)

# print the shape. format is (batch size, token length)
print("input shape: ", inp.shape)

# detokenize the first element
print(detokenize(inp[0]))

input shape:  (8, 256)
1: Надоела зима, скорее бы лето. Люблю его очень 2: Вы работаете? 1: Скорее всего что я работаю Как в же в городе в котором я живу не работать то ? 😊 Ася, вы были в горах? Я вот никогда не была но очень хочу в Тибет, либо на Гималаи, но на худой конец и уральский хребет сойдет 😊 А чем вы занимаетесь? 2: Я медсестра и стендап-комик, люблю животных А вы? У меня 4 собаки 1: Я очень люблю кошек, и у меня одна есть Не много ли собак у вас? 2: Думаю, что нет. Вы замужем? 1: Нет не замужем, у меня же кот есть😀 А ещё я люблю фрукты 2: А я вот вышла замуж за друга)А вы юмористка. 1: Особенно манго и апельсины 2: Любите готовить? 1: Терпеть не могу готовить А юморить моё хобби😊 STOP


In [12]:
m[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Model

In [13]:
def ReformerLM(vocab_size=32000, n_layers=2, mode='train', attention_type=tl.SelfAttention):
    """
    Args: 
        vocab_size (int): size of the vocabulary
        n_layers (int): number of decoder layers
        mode (string): setting of the model which can be 'train', 'eval', or 'predict' 
        attention_type(class): attention class to use 
    Returns: 
        model (TransformerLM): a transformer language model implemented in Trax
    """    
    # initialize an instance of Trax's TransformerLM class
    model = trax.models.ReformerLM( 
        vocab_size=vocab_size,
        # d_model=d_model,
        # d_ff=d_ff,
        n_layers=n_layers,
        # max_len=max_len,
        # dropout = dropout,
        mode = mode,
        attention_type=attention_type
    )
    return model

In [14]:
tmp_model = ReformerLM(n_layers=2, mode='train', attention_type=tl.SelfAttention)
tmp_model.sublayers[-1] = tl.Serial(tl.Dense(vocab_size), tl.LogSoftmax())
print(tmp_model)
del tmp_model

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_32000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
  ]
  Concatenate_in2
  LayerNorm
  Dropout
  Serial[
    Dense_32000
    LogSoftmax
  ]
]


# Train

In [15]:
ChatModel = ReformerLM(vocab_size=32000, n_layers=6, mode='train')
ChatModel.sublayers[-1] = tl.Serial(tl.Dense(vocab_size), tl.LogSoftmax())

In [16]:
def training_loop(ChatModel, train_gen, eval_gen, output_dir = "~/model/"):
    """
    Args:
        ChatModel:  the Reformer language model with LogSoftmax output
        train_gen (generator): train data generator.
        eval_gen (generator): Validation generator. 
        output_dir (string): Path to save the model output. Defaults to './model/'.

    Returns:
        trax.supervised.training.Loop: Training loop for the model.
    """

    output_dir = os.path.expanduser(output_dir)

    # use the warmup_and_rsqrt_decay learning rate schedule
    # lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=4000, max_value=0.0002)

    train_task = training.TrainTask(            
        labeled_data=train_gen,
        loss_layer=tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(2e-4),
        # lr_schedule=lr_schedule,
        n_steps_per_checkpoint=10
    )

    eval_task = training.EvalTask(                      
        labeled_data=eval_gen,
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()]
    )

    loop = training.Loop(ChatModel,
                         train_task,
                         eval_tasks=[eval_task],
                         output_dir=output_dir)
    return loop

In [31]:
!cp /content/drive/MyDrive/model2/model_6_8_02_100.pkl.gz ~/model/model.pkl.gz

In [16]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ../root/model/ # train = name of folder

In [17]:
# !rm -f model/model.pkl.gz
loop = training_loop(ChatModel, train_stream, eval_stream)
loop.run(100)
!cp ~/model/model.pkl.gz /content/drive/MyDrive/model2/rmodel_test.pkl.gz


Step      1: Total number of trainable weights: 69136640
Step      1: Ran 1 train steps in 124.59 secs
Step      1: train CrossEntropyLoss |  10.40490532
Step      1: eval  CrossEntropyLoss |  10.19735813
Step      1: eval          Accuracy |  0.00213068

Step     10: Ran 9 train steps in 370.62 secs
Step     10: train CrossEntropyLoss |  9.49746132
Step     10: eval  CrossEntropyLoss |  8.87853241
Step     10: eval          Accuracy |  0.05470914

Step     20: Ran 10 train steps in 158.22 secs
Step     20: train CrossEntropyLoss |  8.10419369
Step     20: eval  CrossEntropyLoss |  7.84483194
Step     20: eval          Accuracy |  0.04067197

Step     30: Ran 10 train steps in 160.80 secs
Step     30: train CrossEntropyLoss |  7.36917973
Step     30: eval  CrossEntropyLoss |  7.04085541
Step     30: eval          Accuracy |  0.04953999

Step     40: Ran 10 train steps in 159.63 secs
Step     40: train CrossEntropyLoss |  7.06333065
Step     40: eval  CrossEntropyLoss |  7.10729074
Ste

In [ ]:
# !cp ~/model/model.pkl.gz /content/drive/MyDrive/model2/
# !cp ~/model/model.pkl.gz /content/drive/MyDrive/model2/model_6_8_02_20.pkl.gz

## GPU memory footprint

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=fdef2c61bbcead285449bcced893547d3c20f505352fb175df3fa432fc633ea8
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 10.7 GB  | Proc size: 4.2 GB
GPU RAM Free: 568MB | Used: 7043MB | Util  93% | Total 7611MB


# Decoding

## Load the Model

In [ ]:
!cp /content/drive/MyDrive/model2/model.pkl.gz ~/model/model.pkl.gz

In [18]:
# define the `predict_mem_len` and `predict_drop_len` of tl.SelfAttention
def attention(*args, **kwargs):
    # number of input positions to remember in a cache when doing fast inference. 
    kwargs['predict_mem_len'] = 120
    # number of input elements to drop once the fast inference input cache fills up.
    kwargs['predict_drop_len'] = 120
    # return the attention layer with the parameters defined above
    return tl.SelfAttention(*args, **kwargs)

# define the model using the ReformerLM function you implemented earlier.
ChatModel = ReformerLM(
    vocab_size=32000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

# define an input signature so we can initialize our model. shape will be (1, 1) and the data type is int32.
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

In [10]:
# define the model using the ReformerLM function you implemented earlier.
# ChatModel = TransformerLM(n_layers=6, mode='predict')
# ChatModel.sublayers[-1] = tl.Serial(tl.Dense(vocab_size), tl.LogSoftmax())

# define an input signature so we can initialize our model. shape will be (1, 1) and the data type is int32.
# shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

In [19]:
# initialize from file
ChatModel.init_from_file('../root/model/model.pkl.gz', weights_only=True, input_signature=shape11)
# ChatModel.init_from_file('/content/drive/MyDrive/model2/model_6_8_02_100.pkl.gz', weights_only=True, input_signature=shape11)

# save the starting state
STARTING_STATE = ChatModel.state

LayerError: ignored

## Full Text Decoder

In [12]:
def output_gen(ChatModel, start_sentence, temperature, accelerate=True):
    """
    Args:
        ChatModel:  the Transformer language model
        start_sentence (string): starting sentence of the conversation
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """
    
    input_tokens = tokenize(start_sentence)
    
    # Add batch dimension to array
    input_tokens_with_batch = np.expand_dims(input_tokens, axis=0)
    
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        ChatModel,
        inputs=input_tokens_with_batch,
        temperature=temperature,
        accelerate=accelerate
    )

    return output_gen

In [13]:
def generate_dialogue(ChatModel, model_state, start_sentence, max_len, temperature, accelerate):
    """
    Args:
        ChatModel:  the Transformer language model
        model_state (np.array): initial state of the model before decoding
        start_sentence (string): starting sentence of the conversation
        max_len (int): maximum number of tokens to generate 
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """  
    
    delimiter_1 = '1:' 
    delimiter_2 = '2:'
    
    # initialize detokenized output
    sentence = ''
    
    # token counter
    counter = 0

    result = []
    
    # reset the model state when starting a new dialogue
    ChatModel.state = model_state
    
    # calls the output generator
    output = output_gen(ChatModel, start_sentence, temperature=temperature, accelerate=accelerate)
    
    # print the starting sentence
    print(start_sentence.split(delimiter_2)[0])

    control = []
    
    
    # loop below yields the next tokens until max_len is reached. the if-elif is just for prettifying the output.
    for o in output:
        
        result.append(o)
        control.append(int(o))
        
        sentence = detokenize(np.concatenate(result, axis=0))

        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(delimiter_2 + ' ' + sentence)
            sentence = ''
            result.clear()
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(delimiter_1 + ' ' + sentence)
            sentence = ''
            result.clear()

        counter += 1
        
        if counter > max_len:
            break

        if o[0] == 139:
            break


    return detokenize(control), control

In [ ]:
sample_sentence = '1: Привет) 2: Привет!'
control1, control2 = generate_dialogue(ChatModel, model_state=STARTING_STATE, start_sentence=sample_sentence, max_len=200, temperature=0.0, accelerate=True)

In [ ]:
sample_sentence = '1: Как тебя зовут? 2:'
_, _ = generate_dialogue(ChatModel, model_state=STARTING_STATE, start_sentence=sample_sentence, max_len=200, temperature=0.2, accelerate=True)

In [ ]:
sample_sentence = '1: Добрый вечер 2: Доброе утро! 1: Когда увидимся?'
_, _ = generate_dialogue(ChatModel, model_state=STARTING_STATE, start_sentence=sample_sentence, max_len=200, temperature=0.0, accelerate=True)

## Interactive Decoder

In [37]:
def next_symbol(model, input_tokens, temperature=0.0):
    """Returns the next symbol for a given sentence.

    Args:
        input_tokens (list): tokenized sentence
        model (trax.layers.combinators.Serial): The transformer model.

    Returns:
        int: tokenized symbol.
    """
    batch = np.expand_dims(input_tokens, axis=0)

    logits = model(batch)
    # print(logits.shape)
    logits = logits[:, -1, :][0]
    sample = tl.logsoftmax_sample(logits, temperature=temperature)
    # sample = int(np.argmax(logits))
    # print(logits[0])
    
    return sample

In [43]:
ChatModel.state = STARTING_STATE
cur_output_tokens = tokenize('1: Добрый вечер 2: Как')
detokenize([next_symbol(ChatModel, cur_output_tokens)])

'дела'

In [42]:
def interactive_decode_bot_first(model, model_state, greeting='Привет', accelerate=False):
    """Greedy decode function.

    Args:
        model (trax.layers.combinators.Serial): Transformer model.
        model_state: initial state of the model before decoding
        greeting (string):

    Returns:
        string: summary of the input.
    """
    
    delimiter_1 = '1:' 
    delimiter_2 = '2:'
    input_sentence = ''

    cur_output_tokens = tokenize('1: ' + greeting + ' 2:')
    result = cur_output_tokens.copy()[1:]
    cur_output = 0
    stop = 129

    fast_model = tl.Accelerate(model) if accelerate else model

    while (cur_output != stop or input_sentence != 'STOP'):

        # clear past diolog state
        fast_model.state = model_state

        result.append(cur_output)
        sentence = detokenize(result)

        # if sentence.endswith(delimiter_1):
        #     sentence = sentence.split(delimiter_1)[0]
        #     print(delimiter_2 + ' ' + sentence)
        #     sentence = ''
        #     result.clear()
        
        if sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(delimiter_1 + ' ' + sentence)
            sentence = ''
            result.clear()

            input_sentence = input('2: ')
            input_sentence = tokenize(input_sentence)
            cur_output_tokens += input_sentence + [9]
            

        cur_output = next_symbol(fast_model, cur_output_tokens)
        cur_output_tokens.append(cur_output)

    return None

In [ ]:
interactive_decode_bot_first(ChatModel, STARTING_STATE, greeting='Добрый вечер')

In [ ]:
def interactive_decode_bot_second(model, model_state):
    """Greedy decode function.

    Args:
        model (trax.layers.combinators.Serial): Transformer model.
        model_state: initial state of the model before decoding

    Returns: None

    """
    
    delimiter_1 = '1:' 
    delimiter_2 = '2:'

    start_sentence = input('1:')
    cur_output_tokens = tokenize('1: ' + start_sentence + ' 2:')
    result = cur_output_tokens.copy()[1:]
    cur_output = 0
    stop = 129

    while cur_output != stop:

        # clear past diolog state
        model.state = model_state

        result.append(cur_output)
        sentence = detokenize(result)

        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(delimiter_2 + ' ' + sentence)
            sentence = ''
            result.clear()
        
        # if sentence.endswith(delimiter_2):
        #     sentence = sentence.split(delimiter_2)[0]
        #     print(delimiter_1 + ' ' + sentence)
        #     sentence = ''
        #     result.clear()

            input_sentence = input('1: ')
            input_sentence = tokenize(input_sentence)
            cur_output_tokens += input_sentence + [11]
            

        cur_output = next_symbol(model, cur_output_tokens)
        cur_output_tokens.append(cur_output)

    return None

In [ ]:
interactive_decode_bot_second(ChatModel, STARTING_STATE)